<a href="https://colab.research.google.com/github/89CANDY89/R-ANALYSE-john-hopkins/blob/master/clean_data_john.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Importando as bibliotecas necessárias
import pandas as pd
import numpy as np
import requests
import zipfile
import io

In [ ]:
# Baixando e extraindo os dados
url = "https://d396qusza40orc.cloudfront.net/getdata%2Fprojectfiles%2FUCI%20HAR%20Dataset.zip"
response = requests.get(url)
zipfile = zipfile.ZipFile(io.BytesIO(response.content))
zipfile.extractall()

In [ ]:
# Carregando rótulos de atividade e recursos
activity_labels = pd.read_csv('UCI HAR Dataset/activity_labels.txt', sep=' ', header=None, names=['classLabels', 'activityName'])
features = pd.read_csv('UCI HAR Dataset/features.txt', sep=' ', header=None, names=['index', 'featureNames'])
features_wanted = features[features['featureNames'].str.contains('mean\\(\\)|std\\(\\)')]['featureNames'].tolist()
measurements = [column.replace('()', '') for column in features_wanted]

In [ ]:
# Carregando conjuntos de treinamento
train = pd.read_csv('UCI HAR Dataset/train/X_train.txt', sep='\s+', header=None, usecols=features['index'][features['featureNames'].isin(features_wanted)]-1)
train.columns = measurements
train_activities = pd.read_csv('UCI HAR Dataset/train/y_train.txt', sep='\s+', header=None, names=['Activity'])
train_subjects = pd.read_csv('UCI HAR Dataset/train/subject_train.txt', sep='\s+', header=None, names=['SubjectNum'])
train = pd.concat([train_subjects, train_activities, train], axis=1)

In [ ]:
# Carregando conjuntos de teste
test = pd.read_csv('UCI HAR Dataset/test/X_test.txt', sep='\s+', header=None, usecols=features['index'][features['featureNames'].isin(features_wanted)]-1)
test.columns = measurements
test_activities = pd.read_csv('UCI HAR Dataset/test/y_test.txt', sep='\s+', header=None, names=['Activity'])
test_subjects = pd.read_csv('UCI HAR Dataset/test/subject_test.txt', sep='\s+', header=None, names=['SubjectNum'])
test = pd.concat([test_subjects, test_activities, test], axis=1)

In [ ]:
# Mesclando conjuntos de dados
combined = pd.concat([train, test], ignore_index=True)


In [ ]:
# Convertendo classLabels para activityName
combined['Activity'] = combined['Activity'].map(dict(zip(activity_labels['classLabels'], activity_labels['activityName'])))

In [ ]:
# Convertendo SubjectNum para fator
combined['SubjectNum'] = combined['SubjectNum'].astype('category')

In [ ]:
# Derretendo e moldando os dados
combined = pd.melt(combined, id_vars=['SubjectNum', 'Activity'], var_name='variable')
combined = combined.groupby(['SubjectNum', 'Activity', 'variable']).mean().reset_index()


In [ ]:
# Salvando os dados
combined.to_csv('tidyData.txt', index=False)

In [ ]:
dados = pd.read_csv('tidyData.txt')
dados

,SubjectNum,Activity,variable,value
0,1,LAYING,fBodyAcc-mean-X,-0.939099
1,1,LAYING,fBodyAcc-mean-Y,-0.867065
2,1,LAYING,fBodyAcc-mean-Z,-0.882667
3,1,LAYING,fBodyAcc-std-X,-0.924437
4,1,LAYING,fBodyAcc-std-Y,-0.833626
...,...,...,...,...
11875,30,WALKING_UPSTAIRS,tGravityAcc-std-X,-0.954034
11876,30,WALKING_UPSTAIRS,tGravityAcc-std-Y,-0.914934
11877,30,WALKING_UPSTAIRS,tGravityAcc-std-Z,-0.862403
11878,30,WALKING_UPSTAIRS,tGravityAccMag-mean,-0.137628
